# 실험 10-01 증가형 MOSFET의 바이어스 회로

## 공통 코드

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from scipy.signal import savgol_filter # 사비츠키-골레이(Savitzky-Golay) 필터
from numpy import nan, inf

# %matplotlib tk
# %matplotlib inline

def tangent_line(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return lambda t: d*t - d*x + f(x)

def derivative(f, x):
    h = 1e-4
    d = (f(x+h) - f(x-h)) / (2*h)
    return d

def get_simulation_result(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1
    
    with open(file_name, encoding='cp1252') as data_file:
        lines = data_file.read()
        occurrences = lines.count('Step Information:')
        data_file.seek(0)

        line = data_file.readline()
        labels = re.split(', | ,|\t', line)
        labels = [s.strip().upper() for s in labels]

        data = {}

        if (occurrences == 0):
            for label in labels:
                data[label] = []
            
            for line in data_file:
                values = re.split(', | ,|\t', line)
                for i in range(len(values)):
                    value = float(values[i]) * 1000
                    data[labels[i]].append(value)

        else:
            labels_all =[]
            
            lines = data_file.readline() # skip first line starting with 'Step Information:'

            for idx in range(occurrences):
                labels_new = []
                for label in labels:
                    labels_new.append('(%s)@%d' % (label, idx+1))
                labels_all += labels_new

                for label_new in labels_new:
                    data[label_new] = []

                for line in data_file:
                    if (line.startswith('Step Information:')):
                        break
                    values = re.split(', | ,|\t', line)
                    for i in range(len(values)):
                        value = float(values[i]) * 1000
                        data[labels_new[i]].append(value)

            labels = labels_all

        for label in labels:
            data[label] = np.array(data[label][start:end])

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def get_oscilloscpoe_result_tektronix(file_name, start=0, end=-1):
    if end == -1:
        end = None
    else:
        end += 1

    df = pd.read_csv(file_name, header=None, encoding='cp1252',low_memory=False)

    label_ri = df.loc[df.iloc[:,0] == 'Source', 1].index
    ci = 0
    data = {}
    data['TIME'] = df.iloc[start:end, 3].to_numpy().astype(float)
    data['TIME'] -= data['TIME'][0]
    for idx in range(df.shape[1]//6):
        label = df.iloc[label_ri, ci+1].item()
        data[label] = df.iloc[start:end,ci+4].to_numpy().astype(float)
        ci += 6

    # print("labels = ", end='')
    # print(list(data.keys()))
    for label in list(data.keys()):
        print("data['%s'] : sample number = %d" % (label, len(data[label])))

    return data

def print_array(label, values):
    print('%s = [' % label, end='')
    for idx, vd in enumerate(values):
        print('{:11.3f}'.format(vd), end='')
        if (idx+1 != len(values)):
            print(', ', end='')
    print(']')

def print_value(label, value):
    print('%s = ' % label, end='')
    print('{:11.3f}'.format(value))

def draw_plot(xs, ys, label, style_idx, marker_num=16, scatter=False, scatter_s=2):
    linestyle  = ['-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-', 
                  '-',          '-',            '-',            '-',            '-']
    colors     = ['blue',       'green',        'red',          'orange',       'purple',
                  'cyan',       'darkseagreen', 'brown',        'goldenrod',    'darkviolet',
                  'steelblue',  'limegreen',    'tomato',       'tan',          'deeppink',
                  'navy',       'lightgreen',   'indianred',    'khaki',        'rebeccapurple',
                  'slategray',  'forestgreen',  'orangered',    'wheat',        'orchid']
    markers    = ['o',          'v',            '<',            's',            'p',
                  'h',          '*',            'X',            'x',            '^',
                  '>',          'P',            'D',            'H',            'd',
                  '|',          '-',            '4'             '5',            '6',
                  '7',          '8',            '9 ',           '10',           '+']

    if isinstance(xs, list):
        xs = np.array(xs)
    if isinstance(ys, list):
        ys = np.array(ys)

    if (marker_num == 0):
        selected_markevery = None
        selected_marker = None
    elif not isinstance(xs, np.ndarray):
        selected_markevery = None
        selected_marker = markers[style_idx]
    elif (marker_num > 0):
        markeverys = []
        if xs[0] > xs[-1]:
            first_idx = -1
        else:
            first_idx = 0
        x_step = (xs.max() - xs.min()) / marker_num
        for idx in range(len(markers)):
            x_start = xs.min() + ((x_step / (len(markers) + 1)) * idx)
            marker_list = []
            for j in range(marker_num):
                tmp_array = np.where(xs > (x_start + (x_step * j)))
                if (len(tmp_array[0]) > 0):
                    marker_list.append(tmp_array[0][first_idx])
            markeverys.append(marker_list)

        selected_markevery = markeverys[style_idx]
        selected_marker = markers[style_idx]
    else:
        selected_markevery = None
        selected_marker = markers[style_idx]

    if (scatter):
        plt.scatter(xs, ys, color=colors[style_idx], marker=selected_marker, s=scatter_s, label=label)
    else:
        plt.plot(xs, ys, ls=linestyle[style_idx], color=colors[style_idx], marker=selected_marker, markevery=selected_markevery, label=label)
    if (label != None):
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

def set_plot_size(xrate, yrate):
    if (len(set_plot_size.g_plt_figsize) == 0):
        set_plot_size.g_plt_figsize = plt.rcParams["figure.figsize"]
    size = []
    size.append(set_plot_size.g_plt_figsize[0] * xrate)
    size.append(set_plot_size.g_plt_figsize[1] * yrate)
    plt.figure(figsize=size)

set_plot_size.g_plt_figsize = []

## LTspice 명령

* 컴포넌트 추가 명령
    + Voltage source: Component -> voltage (V)
    + Resister: Component -> res (R)
    + Diode: diode (D)
    + Zener diode: Component -> zenor
    + Bipolar NPN transistor: Component -> npn
    + Bipolar PNP transistor: Component -> pnp
    + N-Channel MOSFET transistor: Component -> nmos
    + Ground: ground (G)

&nbsp;

* SPICE 시뮬레이션 설정 명령
    * DC Sweep analysis
        + Compute the DC operating point over swept independent source(s).
        + Syntax: .dc \<source1\> \<start1\> \<stop1\> [\<increment1\>] [\<source2\> ...]
            + source: source name
            + start: start value
            + stop: stop value
            + increment: increment value
    * Transient analysis
        + Performa time-domain analysis
        + Syntax: .tran \<Tprint\> \<Tstop\> [\<Tstart\> [\<Tmaxstep\>]]
            - Tprint: printing increment (Use 0 in LTspice)
            - Tstop: stop time
            - Tstart: time to start saving waveform data
            - Tmaxstep: maximum timestep size
    * Parametric analysis (list)
        + Performs a parametric sweep for all of the analysis
        + Syntax: .step param \<name\> list \<value1\> [\<value2\> [\<value3\> [...]]]
            - name: name of parameter to step
    + DC Bias Point  analysis
        + Performa DC bias point analysis
        + Syntax: .op

&nbsp;

* 기타 명령
    + 회전 (Control + R)
    + Wire (F3)
    + Move (F7)
    + Drag (F8)
    + Delete (F5)
    + Duplicate (F6)


*******************************************************************************
## 시뮬레이션 1-1 결과 분석

## 회로도

![Markdown-mark](../ltspice/simulation_01_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_01_01.txt')

### 2. 시뮬레이션 결과에서 $V_{GSQ}$, $I_{DQ}$, $V_{DSQ}$를 가져온다.

In [ ]:
V_DD_V      = 15.
R_D_kOhm_a  = data['RDVAR'] / 1000000.
V_GSQ_V_a   = data['V(VG)'] / 1000.
I_DQ_mA_a   = data['ID(M1)']
V_DSQ_V_a   = data['V(VD)'] / 1000.

print_array('R_D_kOhm             ', R_D_kOhm_a)
print_array('V_GSQ_V              ', V_GSQ_V_a)
print_array('I_DQ_mA              ', I_DQ_mA_a)
print_array('V_DSQ_V              ', V_DSQ_V_a)

print('')
V_Tn_V = 1.730
V_GS_m_Tn_V = V_GSQ_V_a - V_Tn_V
V_DS_m_GS_m_Tn_V = V_DSQ_V_a - V_GS_m_Tn_V
print_array('V_GS -  V_Tn         ', V_GS_m_Tn_V)
print_array('V_DS - (V_GS - V_Tn) ', V_DS_m_GS_m_Tn_V)

### 3. $V_{DSQ}$ - $I_{DQ}$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_DSQ_V')
plt.ylabel('I_DQ_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 16])
plt.ylim([-1, 210])
for idx in range(len(R_D_kOhm_a)):
    draw_plot(V_DSQ_V_a[idx], I_DQ_mA_a[idx], '(R_D_kOhm=%1.3f)' % R_D_kOhm_a[idx], idx, scatter=True, scatter_s=20)
plt.show()

### 4. 부하선을 그린다.

In [ ]:
plt.xlabel('V_DSQ_V')
plt.ylabel('I_DQ_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 16])
plt.ylim([-1, 210])
for idx in range(len(R_D_kOhm_a)):
    draw_plot([0, V_DD_V], [V_DD_V/R_D_kOhm_a[idx], 0], None, idx, marker_num = 0)
    draw_plot(V_DSQ_V_a[idx], I_DQ_mA_a[idx], '(R_D_kOhm=%1.3f)' % R_D_kOhm_a[idx], idx, scatter=True, scatter_s=20)
plt.show()

*******************************************************************************
## 시뮬레이션 1-2 결과 분석

## 회로도

![Markdown-mark](../ltspice/simulation_01_02.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_01_02.txt')

### 2. 시뮬레이션 결과에서 $I_{DQ}$, $V_{DSQ}$를 가져온다.

In [ ]:
I_DQ_mA_a_dc_sweep = data['ID(M1)']
V_DSQ_V_a_dc_sweep = data['V(VD)'] / 1000.

V_GSQ_V = 2.5

### 3. $V_{DSQ}$ - $I_{DQ}$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_DSQ_V')
plt.ylabel('I_DQ_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 16])
plt.ylim([-1, 210])
draw_plot(V_DSQ_V_a_dc_sweep, I_DQ_mA_a_dc_sweep, '(V_GSQ_V=%1.3f)' % V_GSQ_V, 10, marker_num=0)
plt.show()

### 4. 시뮬레이션 1-1의 결과와 비교한다.

In [ ]:
plt.xlabel('V_DSQ_V')
plt.ylabel('I_DQ_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 16])
plt.ylim([-1, 210])
for idx in range(len(R_D_kOhm_a)):
    draw_plot([0, V_DD_V], [V_DD_V/R_D_kOhm_a[idx], 0], None, idx, marker_num = 0)
    draw_plot(V_DSQ_V_a[idx], I_DQ_mA_a[idx], '(R_D_kOhm=%1.3f)' % R_D_kOhm_a[idx], idx, scatter=True, scatter_s=20)
draw_plot(V_DSQ_V_a_dc_sweep, I_DQ_mA_a_dc_sweep, '(V_GSQ_V=%1.3f)' % V_GSQ_V, 10, marker_num=0)
plt.show()

*******************************************************************************
## 시뮬레이션 2-1 결과 분석

## 회로도

![Markdown-mark](../ltspice/simulation_02_01.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_02_01.txt')

### 2. 시뮬레이션 결과에서 $V_{GSQ}$, $I_{DQ}$, $V_{DSQ}$를 가져온다.

In [ ]:
V_DD_V      = 15.
R_D_kOhm    = 0.100
R_S_kOhm_a  = data['RSVAR'] / 1000000.
V_GSQ_V_a   = (data['V(VG)'] - data['V(VS)']) / 1000.
I_DQ_mA_a   = data['ID(M1)']
V_DSQ_V_a   = (data['V(VD)'] - data['V(VS)']) / 1000.

print_array('R_S_kOhm             ', R_S_kOhm_a)
print_array('V_GSQ_V              ', V_GSQ_V_a)
print_array('I_DQ_mA              ', I_DQ_mA_a)
print_array('V_DSQ_V              ', V_DSQ_V_a)

print('')
V_Tn_V = 1.730
V_GS_m_Tn_V = V_GSQ_V_a - V_Tn_V
V_DS_m_GS_m_Tn_V = V_DSQ_V_a - V_GS_m_Tn_V
print_array('V_GS - V_Tn          ', V_GS_m_Tn_V)
print_array('V_DS - (V_GS - V_Tn) ', V_DS_m_GS_m_Tn_V)

### 3. $V_{DSQ}$ - $I_{DQ}$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_DSQ_V')
plt.ylabel('I_DQ_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 16])
plt.ylim([-1, 210])
for idx in range(len(R_D_kOhm_a)):
    draw_plot(V_DSQ_V_a[idx], I_DQ_mA_a[idx], '(R_S_kOhm=%1.3f)' % R_S_kOhm_a[idx], idx, scatter=True, scatter_s=20)
plt.show()

### 4. 부하선을 그린다.

In [ ]:
plt.xlabel('V_DSQ_V')
plt.ylabel('I_DQ_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 16])
plt.ylim([-1, 210])
for idx in range(len(R_S_kOhm_a)):
    draw_plot([0, V_DD_V], [V_DD_V/(R_D_kOhm + R_S_kOhm_a[idx]), 0], None, idx, marker_num = 0)
    draw_plot(V_DSQ_V_a[idx], I_DQ_mA_a[idx], '(R_S_kOhm=%1.3f)' % R_S_kOhm_a[idx], idx, scatter=True, scatter_s=20)
plt.show()

*******************************************************************************
## 시뮬레이션 2-2 결과 분석

## 회로도

![Markdown-mark](../ltspice/simulation_02_02.png)

### 1. 시뮬레이션 결과 파일을 읽는다.

In [ ]:
data = get_simulation_result('../ltspice/simulation_02_02.txt')

### 2. 시뮬레이션 결과에서 $I_{DQ}$, $V_{DSQ}$를 가져온다.

In [ ]:
V_GSQ_V_a_dc_sweep = np.array([      3.057,       2.754,       2.562,       2.391,       2.292,       2.180])
V_DSQ_V_a_dc_sweep = []
I_DQ_mA_a_dc_sweep = []

for idx in range(len(V_GSQ_V_a_dc_sweep)):
    V_DSQ_V_a_dc_sweep.append(data['(V(VD))@%d' % (idx+1)] / 1000.)
    I_DQ_mA_a_dc_sweep.append(data['(ID(M1))@%d' % (idx+1)])

### 3. $V_{DSQ}$ - $I_{DQ}$ 그래프를 그린다.

In [ ]:
plt.xlabel('V_DSQ_V')
plt.ylabel('I_DQ_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 16])
plt.ylim([-1, 210])
for idx in range(len(V_GSQ_V_a_dc_sweep)):
    draw_plot(V_DSQ_V_a_dc_sweep[idx], I_DQ_mA_a_dc_sweep[idx], '(V_GSQ_v=%1.3f)' % V_GSQ_V_a_dc_sweep[idx], idx, marker_num=0)
plt.show()

### 4. 시뮬레이션 2-1의 결과와 비교한다.

In [ ]:
plt.xlabel('V_DSQ_V')
plt.ylabel('I_DQ_mA')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlim([-0.5, 16])
plt.ylim([-1, 210])
for idx in range(len(R_S_kOhm_a)):
    draw_plot([0, V_DD_V], [V_DD_V/(R_D_kOhm + R_S_kOhm_a[idx]), 0], None, idx, marker_num = 0)
    draw_plot(V_DSQ_V_a[idx], I_DQ_mA_a[idx], '(R_S_kOhm=%1.3f)' % R_S_kOhm_a[idx], idx, scatter=True, scatter_s=20)
for idx in range(len(V_GSQ_V_a_dc_sweep)):
    draw_plot(V_DSQ_V_a_dc_sweep[idx], I_DQ_mA_a_dc_sweep[idx], '(V_GSQ_v=%1.3f)' % V_GSQ_V_a_dc_sweep[idx], idx, marker_num=0)
plt.show()